# T20 Upset Radar and Strategy Simulator: Final Story

This notebook is the portfolio-ready narrative for the MVP.

It walks through:
1. Data scope and quality snapshot
2. Modeling and calibration baseline
3. Upset-focused evaluation
4. Explainability and local upset narratives
5. Strategy simulator examples


In [ ]:
from __future__ import annotations

import json
from pathlib import Path

import pandas as pd
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

from src.config import PROCESSED_DIR
from src.data_prep import (
    assign_favorite_underdog_from_elo,
    build_team1_win_target,
    load_matches,
    time_based_split,
)
from src.explain import build_counterfactual_explanation, rank_notable_upsets
from src.features import build_pre_match_feature_frame
from src.models import calibrate_classifier, evaluate_binary_model, train_logistic_baseline
from src.simulation import ScenarioInput, build_scenario_features, score_scenario


## 1) Data Scope and Quality Snapshot

In [ ]:
df = load_matches()
df = build_team1_win_target(df)
df = assign_favorite_underdog_from_elo(df)

quality_path = PROCESSED_DIR / "data_quality_report.json"
quality = {}
if quality_path.exists():
    quality = json.loads(quality_path.read_text())

print("Rows:", len(df))
print("Date range:", df["date"].min(), "->", df["date"].max())
print("Unique teams:", df["team1"].nunique())
print("Overall upset rate:", round(float(df["is_upset"].mean()), 4))
if quality:
    print("Duplicate match_ids:", quality.get("duplicate_match_id_count", "n/a"))
    print("Columns with missing values:", quality.get("columns_with_missing_values", "n/a"))


## 2) Baseline Modeling and Calibration

In [ ]:
train_df, valid_df, test_df = time_based_split(df)

X_train, y_train = build_pre_match_feature_frame(train_df)
X_valid, y_valid = build_pre_match_feature_frame(valid_df)
X_test, y_test = build_pre_match_feature_frame(test_df)

base_model = train_logistic_baseline(X_train, y_train)
calibrated_model = calibrate_classifier(base_model, X_valid, y_valid)
test_metrics = evaluate_binary_model(calibrated_model, X_test, y_test)
pd.Series(test_metrics).round(4)


## 3) Upset-Focused Evaluation

In [ ]:
test_eval = test_df.copy()
test_eval = assign_favorite_underdog_from_elo(test_eval)
X_eval, _ = build_pre_match_feature_frame(test_eval)
test_eval["team1_win_prob"] = calibrated_model.predict_proba(X_eval)[:, 1]
test_eval["pred_team1_win"] = (test_eval["team1_win_prob"] >= 0.5).astype(int)

# Predict winner names in match orientation.
test_eval["pred_winner"] = test_eval.apply(
    lambda r: r["team1"] if r["pred_team1_win"] == 1 else r["team2"], axis=1
)
test_eval["pred_is_upset"] = (test_eval["pred_winner"] == test_eval["underdog_team"]).astype(int)

upset_cm = confusion_matrix(test_eval["is_upset"], test_eval["pred_is_upset"])
upset_prf = precision_recall_fscore_support(
    test_eval["is_upset"], test_eval["pred_is_upset"], average="binary", zero_division=0
)

print("Upset confusion matrix (rows=true, cols=pred):")
print(upset_cm)
print("Upset precision/recall/f1:", tuple(round(float(x), 4) for x in upset_prf[:3]))


## 4) Explainability: Notable Upsets + Counterfactual View

In [ ]:
notable = rank_notable_upsets(df, top_n=8)
notable[["date", "team1", "team2", "winner", "elo_diff"]].head(8)


In [ ]:
if len(notable) > 0:
    top_case = notable.iloc[0]
    case_df = df[(df["team1"] == top_case["team1"]) & (df["team2"] == top_case["team2"])].copy()
    cols = [
        "team1", "team2", "match_stage", "venue", "toss_winner", "toss_decision",
        "elo_team1", "elo_team2", "elo_diff", "team1_form_5", "team2_form_5",
        "team1_form_10", "team2_form_10", "h2h_win_pct"
    ]
    if not case_df.empty:
        local_exp = build_counterfactual_explanation(calibrated_model, case_df[cols].head(1))
        print("Base team1 win probability:", round(local_exp["base_team1_win_prob"], 4))
        pd.DataFrame(local_exp["counterfactuals"])
    else:
        print("No direct oriented row found for the top upset case.")
else:
    print("No notable upsets found in current filtered data.")


## 5) Strategy Simulator Example Scenarios

In [ ]:
teams = sorted(df["team1"].dropna().unique())
venues = sorted(df["venue"].dropna().unique())

team1 = teams[0]
team2 = teams[1]
venue = venues[0]

scenario_a = ScenarioInput(
    team1=team1,
    team2=team2,
    match_stage="Group Stage",
    venue=venue,
    toss_winner=team1,
    toss_decision="bat",
    elo_team1=1200,
    elo_team2=1175,
    team1_form_5=0.6,
    team2_form_5=0.5,
    team1_form_10=0.58,
    team2_form_10=0.52,
    h2h_win_pct=0.55,
)
scenario_b = ScenarioInput(**{**scenario_a.__dict__, "toss_decision": "field"})

res_a = score_scenario(calibrated_model, build_scenario_features(scenario_a))
res_b = score_scenario(calibrated_model, build_scenario_features(scenario_b))

pd.DataFrame([
    {"scenario": "bat first", **res_a},
    {"scenario": "field first", **res_b},
])


## Conclusions and MVP Status

- The project now supports calibrated win probabilities, upset-risk labeling, and scenario-based strategy comparison.
- Explainability has both global and local pathways for communication and trust.
- Remaining MVP polish focuses on curated upset narratives and presentation assets (screenshots/GIF + results summary table).


## 6) Missed Upsets Audit (High-Confidence Misses)

This table highlights matches where the model gave the favorite a high win probability, but the underdog actually won.

Interpretation focus:
- larger `favorite_confidence` means the model was more surprised by the upset
- these are useful cases for feature-gap analysis and future model improvements

In [ ]:
audit_df = test_eval.copy()
audit_df["actual_winner"] = audit_df["winner"]
audit_df["favorite_confidence"] = audit_df.apply(
    lambda r: r["team1_win_prob"] if r["favorite_team"] == r["team1"] else (1 - r["team1_win_prob"]),
    axis=1,
)
audit_df["is_missed_upset"] = (audit_df["is_upset"] == 1) & (audit_df["pred_is_upset"] == 0)

missed_upsets = audit_df[audit_df["is_missed_upset"]].copy()
missed_upsets = missed_upsets.sort_values("favorite_confidence", ascending=False)

cols = [
    "date",
    "team1",
    "team2",
    "actual_winner",
    "favorite_team",
    "underdog_team",
    "favorite_confidence",
    "team1_win_prob",
    "is_upset",
    "pred_is_upset",
    "match_stage",
    "venue",
]

missed_upsets[cols].head(15)

## 7) Curated Upset Narratives (Top Cases)

The table below converts top upset cases into short portfolio-style narratives.

Selection logic:
- rank by absolute ELO gap among upset matches
- keep concise fields for storytelling (`stage`, `venue`, `favorite`, `winner`, `elo_gap`)
- auto-generate a short explanation line for each case

In [ ]:
narrative_cases = rank_notable_upsets(df, top_n=10).copy()

if narrative_cases.empty:
    print("No upset cases found for narrative generation.")
else:
    narrative_cases = narrative_cases.copy()
    narrative_cases["elo_gap"] = narrative_cases["elo_diff"].abs().round(1)

    def build_case_narrative(row: pd.Series) -> str:
        favorite = row.get("favorite_team", "favorite")
        winner = row.get("winner", "winner")
        stage = row.get("match_stage", "Unknown Stage")
        toss_decision = row.get("toss_decision", "unknown")
        gap = row.get("elo_gap", "n/a")
        return (
            f"At {stage}, {winner} beat favorite {favorite} despite an ELO gap of {gap}. "
            f"Toss decision was '{toss_decision}', suggesting context pressure beyond baseline strength."
        )

    narrative_cases["narrative"] = narrative_cases.apply(build_case_narrative, axis=1)

    narrative_cols = [
        "date",
        "team1",
        "team2",
        "winner",
        "favorite_team",
        "underdog_team",
        "elo_gap",
        "match_stage",
        "venue",
        "narrative",
    ]
    narrative_cases[narrative_cols].head(10)

## 8) Cross-Case Patterns from Curated Upsets

This quick summary aggregates the curated upset set to surface recurring contexts (stage, toss decision, venues).

In [ ]:
if "narrative_cases" in globals() and len(narrative_cases) > 0:
    stage_summary = narrative_cases["match_stage"].value_counts(dropna=False).rename("count").to_frame()
    toss_summary = narrative_cases["toss_decision"].value_counts(dropna=False).rename("count").to_frame()

    print("Curated upset count:", len(narrative_cases))
    print("\nTop stages in curated upsets:")
    display(stage_summary.head(10))
    print("\nToss decisions in curated upsets:")
    display(toss_summary)
else:
    print("Narrative cases not available.")